In [72]:
from pymtl3 import *
from pymtl3.passes.backends.verilog import *

In [6]:
a = ["","X", "O", "-"]
for i in range(4**7):
    print("{}{}{}{}{}{}{}{}{}".format("X" if i%2 else "O", i%4, i//4%4, i//16%4, "X" if i%2 else "O", i//64%4, i//256%4, i//1024%4, "X" if i%2 else "O"))

O000O000O
X100X000X
O200O000O
X300X000X
O010O000O
X110X000X
O210O000O
X310X000X
O020O000O
X120X000X
O220O000O
X320X000X
O030O000O
X130X000X
O230O000O
X330X000X
O001O000O
X101X000X
O201O000O
X301X000X
O011O000O
X111X000X
O211O000O
X311X000X
O021O000O
X121X000X
O221O000O
X321X000X
O031O000O
X131X000X
O231O000O
X331X000X
O002O000O
X102X000X
O202O000O
X302X000X
O012O000O
X112X000X
O212O000O
X312X000X
O022O000O
X122X000X
O222O000O
X322X000X
O032O000O
X132X000X
O232O000O
X332X000X
O003O000O
X103X000X
O203O000O
X303X000X
O013O000O
X113X000X
O213O000O
X313X000X
O023O000O
X123X000X
O223O000O
X323X000X
O033O000O
X133X000X
O233O000O
X333X000X
O000O100O
X100X100X
O200O100O
X300X100X
O010O100O
X110X100X
O210O100O
X310X100X
O020O100O
X120X100X
O220O100O
X320X100X
O030O100O
X130X100X
O230O100O
X330X100X
O001O100O
X101X100X
O201O100O
X301X100X
O011O100O
X111X100X
O211O100O
X311X100X
O021O100O
X121X100X
O221O100O
X321X100X
O031O100O
X131X100X
O231O100O
X331X100X
O002O100O
X102X100X
O202O100O
X302X100X


In [73]:
# Simulation
class Adder(Component):
    def construct (s):
        s.a = InPort(32)
        s.b = InPort(32)
        s.q = OutPort(32)

        @update
        def comb():
            s.q @= s.a + s.b

In [74]:
a = Adder()
a.apply(DefaultPassGroup(linetrace=True, textwave=True))
a.sim_reset()

a.a @= 5
a.b @= 7
a.sim_eval_combinational()
print(a.q.uint())

12


In [75]:
a.sim_tick()
a.print_textwave()


      |0    |1    |2    |3    

  clk /‾‾\__/‾‾\__/‾‾\__/‾‾\__

reset ‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾\_____

    a  0000000          |+0005

    b  0000000          |+0007

    q  0000000          |+000c

